In [2]:
# Import necessary library
from pypdf import PdfReader

import os
from dotenv import load_dotenv
from openai import OpenAI

from IPython.display import display, Markdown
import gradio as gr

In [3]:
# Read the pdf file
linkedin_text = ''
file_name = './me/linkedin.pdf'

reader = PdfReader(file_name)

for page_num in range(len(reader.pages)):
    page = reader.pages[page_num]
    text = page.extract_text()
    if text:
        linkedin_text += text + '\n'

In [4]:
# Read the summary file
summary_text = ''
file_name = './me/summary.txt'

try:
    with open(file_name, 'r') as f:
        summary_text = f.read()
except FileNotFoundError:
    print('File not found')
except Exception as e:
    print(f'Error: {e}')

In [30]:
# Write a system prompt
name = 'Ed Donner'

# Write a system prompt instructing what to do.
# Add to the prompt the resume and summary text.
system_prompt = f'You are acting as {name}.'
system_prompt = f'You are answering questions on {name} website particularly questions regarding {name} career, background, and experience.'
system_prompt = f'Your responsibility is to represent {name} for interactions on the website as faithfully as possible.'
# system_prompt = f'Be professional and engaging, as if talking to future employer or potential client.'
system_prompt = f'Be cute and playful, as if talking to a friend or an acquaintance.'
system_prompt += f'If you do not know the answer, just say so. Do not invention the answer.'
system_prompt += f'Here is some information about {name} that you may refer to in answering those questions.\n\n'
system_prompt += f'\n\nHere is {name} Linkedin information. \n\n {linkedin_text}\n\n'
system_prompt += f'\n\nSome more information about {name} is here.\n\n {summary_text}\n\n'
system_prompt += f'With this context, chat with the user. Always stay in character as {name}.'


In [6]:
# Set up client
load_dotenv()

client = OpenAI(api_key=os.getenv('DEEPSEEK_API_KEY'), base_url='https://api.deepseek.com/v1')

In [7]:
# Write a chat function for Gradio to interact with the user
def chat(message, history):
    messages = [{'role': 'system', 'content': system_prompt}] + history + [{'role': 'user', 'content': message}]
    response = client.chat.completions.create(
        model='deepseek-chat',
        messages=messages
    )

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type='messages').launch()#share=True)

In [ ]:
# setting up evaluator prompt
evaluator_prompt = f"You are to evaluate whether a response to a question is acceptable."
evaluator_prompt = f"The question was from a User.  The response was from an Agent."
evaluator_prompt += f"The Agent was instructed to play to a role of {name} on {name} website."
evaluator_prompt += f"The Agent was instructed to be cute and playful, as if talking to a friend or an acquaintance."
evaluator_prompt += f"The Agent has been provide with context on {name} in a form of summary and Linkedin details."
evaluator_prompt += f"Here is the information. Summary: \n{summary_text}.\nLinkedin: \n{linkedin_text}."
evaluator_prompt += f"With this context, please evaluate the latest response, replying whether the response is acceptable and your feedback."

In [9]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here is the conversation between the User and Agent: \n\n{history}\n"
    user_prompt += f"Here is the latest message from User: \n\n{message}\n"
    user_prompt += f"Here is the latest reply from Agent: \n\n{reply}\n"
    user_prompt += f"Please evaluate the latest response, replying whether the response is acceptable and your feedback."
    return user_prompt

In [13]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [11]:
gemini = OpenAI(
    api_key=os.getenv('GOOGLE_API_KEY'), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )


In [26]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(
        model='gemini-2.0-flash',
        messages=messages,
        response_format=Evaluation
    )
    return response.choices[0].message.parsed

In [27]:
# Test
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Do you hold a patent?"}
    ]
response = client.chat.completions.create(
    model = 'gpt-4.1-nano',
    messages=messages
)

In [28]:
reply = response.choices[0].message.content

In [29]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is correct. The agent correctly identifies that Ed Donner has a patent for an apparatus for determining role fitness while eliminating unwanted bias. The response is also a professional tone.')

In [53]:
messages[:1]

[{'role': 'system',
  'content': "Be professional, courteous, and avoid playful remarks which may be misinterpreted.If you do not know the answer, just say so. Do not invention the answer.Here is some information about Ed Donner that you may refer to in answering those questions.\n\n\n\nHere is Ed Donner Linkedin information. \n\n \xa0 \xa0\nContact\ned.donner@gmail.com\nwww.linkedin.com/in/eddonner\n(LinkedIn)\nedwarddonner.com (Personal)\nTop Skills\nCTO\nLarge Language Models (LLM)\nPyTorch\nPatents\nApparatus for determining role\nfitness while eliminating unwanted\nbias\nEd Donner\nCo-Founder & CTO at Nebula.io, repeat Co-Founder of AI startups,\nspeaker & advisor on Gen AI and LLM Engineering\nNew York, New York, United States\nSummary\nI’m a technology leader and entrepreneur. I'm applying AI to a field\nwhere it can make a massive impact: helping people discover their\npotential and pursue their reason for being. But at my core, I’m a\nsoftware engineer and a scientist. I learn

In [52]:
test_list = [1,2,3,4]
test_list[:1]

[1]